# **ETL: Limpieza del conjunto de datos**

## **Líbrerias y módulos necesarios**

In [450]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
warnings.filterwarnings('ignore')

## **Conjunto de datos 1: Cáncer de mama**

```{note}
La información utilizada para este estudio está basada en el Certificado de Defunción del Ministerio de Salud de Colombia ``{cite}`minsalud_defuncion` ``. En este caso, observaciones sobre casos de defunción relacionados con el cáncer de mama. 

```

Inicialmente, se importa el primer conjunto de datos que contiene información detallada sobre la defunción de pacientes debido al cáncer de mama.

In [451]:
url = r"C:\Users\kamac\OneDrive - Universidad del Norte\UNINORTE\SEMESTRE VII. 2025 - 10\U - Seminario investigativo\Proyecto Final\Datos\data.xlsx"
data = pd.read_excel(url, na_values = [" "])

Veamos las primeras cinco observaciones para realizar un análisis inicial del conjunto de datos.

In [452]:
data.head()

,COD_DPTO,cod_munic,a_defun,sit_defun,cod_inst,nom_inst,ano,mes,sexo,est_civil,...,causa_666,cau_homol,EDAD,llave,CODPTO,filter_$,AÑOL,MAMA,quinquenio,REGION
0,5,1,1,1,5.001021e+10,HOSPITAL PABLO TOBON URIBE,2014,7,2,1,...,208.0,24,NaN,0500111201407 21103,5,1,a,mama,2010a201,1
1,5,45,1,1,NaN,NaN,1997,8,2,5,...,NaN,24,NaN,0504511199708 25109,5,0,a,mama,1995a199,1
2,5,266,1,2,NaN,NaN,1986,4,2,1,...,NaN,24,NaN,0526612198604 21109,5,0,a,mama,1985a198,1
3,8,1,1,1,NaN,NaN,1989,7,2,2,...,NaN,24,NaN,0800111198907 22109,8,0,a,mama,1985a198,2
4,8,1,1,1,NaN,NaN,1998,8,2,2,...,208.0,24,10.0,0800111199808 22103,8,1,a,mama,1995a199,2


Las primeras cinco observaciones muestran datos con valores faltantes en varias columnas, diferencias en la codificación de variables y posibles inconsistencias en los identificadores de municipios e instituciones de salud. 

In [453]:
data.shape

(46869, 53)

El conjunto de datos inicial cuenta con **53** columnas (variables) y **46869** filas (observaciones).

In [454]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46869 entries, 0 to 46868
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   COD_DPTO    46869 non-null  int64  
 1   cod_munic   46869 non-null  int64  
 2   a_defun     46869 non-null  int64  
 3   sit_defun   46869 non-null  int64  
 4   cod_inst    19155 non-null  float64
 5   nom_inst    19316 non-null  object 
 6   ano         46869 non-null  int64  
 7   mes         46869 non-null  int64  
 8   sexo        46869 non-null  int64  
 9   est_civil   46869 non-null  int64  
 10  gru_ed1     46869 non-null  int64  
 11  nivel_edu   46869 non-null  int64  
 12  MUERTEPORO  16875 non-null  float64
 13  SIMUERTEPO  917 non-null    float64
 14  OCUPACION   11341 non-null  object 
 15  IDPERTET    16875 non-null  float64
 16  codpres     29361 non-null  float64
 17  codptore    46869 non-null  int64  
 18  codmunre    46869 non-null  int64  
 19  area_res    33952 non-nul

A primera vista, observamos que las variables tienen nombres poco descriptivos, lo que dificulta la comprensión de su significado. Además, muchas de ellas están representadas como valores numéricos de tipo *float64*, aunque en realidad corresponden a categorías. Por ello, procederemos a redefinir los tipos de datos en función del significado de cada variable. También identificamos una alta presencia de datos faltantes en algunas columnas (incluyendo dos columnas que en su totalidad son datos faltantes), lo que requerirá un tratamiento adecuado.

In [455]:
column_types = {"COD_DPTO": "object", "cod_munic": "object", "a_defun": "object", "sit_defun": "object", "cod_inst": "Int64", "nom_inst": "object", 
                "ano": "Int64", "mes": "object", "sexo": "object", "est_civil": "object", "gru_ed1": "object", "nivel_edu": "object", 
                "MUERTEPORO": "object", "SIMUERTEPO": "object", "OCUPACION": "object", "IDPERTET": "object", "codpres": "object", 
                "codptore": "object", "codmunre": "object", "area_res": "object", "seg_social": "object", "IDADMISALU": "object", 
                "IDCLASADMI": "object", "pman_muer": "object", "cons_exp": "object", "man_muer": "object", "c_muerte": "object", 
                "asis_med": "object", "IDPROFCER": "object", "causa_666": "object", "cau_homol": "object", "filter_$": "object",
                "quinquenio": "object", "REGION": "object", 'CODPTO': 'object'}


In [456]:
data = pd.read_excel(url, na_values = [" "], dtype = column_types)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46869 entries, 0 to 46868
Data columns (total 53 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   COD_DPTO    46869 non-null  object 
 1   cod_munic   46869 non-null  object 
 2   a_defun     46869 non-null  object 
 3   sit_defun   46869 non-null  object 
 4   cod_inst    19155 non-null  Int64  
 5   nom_inst    19316 non-null  object 
 6   ano         46869 non-null  Int64  
 7   mes         46869 non-null  object 
 8   sexo        46869 non-null  object 
 9   est_civil   46869 non-null  object 
 10  gru_ed1     46869 non-null  object 
 11  nivel_edu   46869 non-null  object 
 12  MUERTEPORO  16875 non-null  object 
 13  SIMUERTEPO  917 non-null    object 
 14  OCUPACION   11341 non-null  object 
 15  IDPERTET    16875 non-null  object 
 16  codpres     29361 non-null  object 
 17  codptore    46869 non-null  object 
 18  codmunre    46869 non-null  object 
 19  area_res    33952 non-nul

Nótese que el cambio se ha realizado exitosamente.

### **Datos Faltantes**

Observemos a detalle los datos faltantes en este conjunto de datos.

In [457]:
missing_percent = (data.isnull().sum() / len(data)) * 100
missing_percent = missing_percent.sort_values(ascending=False)
missing_percent[missing_percent > 0]

codocur       100.000000
codmunoc      100.000000
c_mcm1         99.850648
C_ANT32        99.639421
C_DIR12        99.372720
C_ANT22        98.975869
C_ANT12        98.598221
C_PAT2         98.480872
SIMUERTEPO     98.043483
man_muer       93.353816
c_ant3         85.704837
c_pat1         84.819390
OCUPACION      75.802769
IDCLASADMI     65.582795
IDADMISALU     65.452645
MUERTEPORO     63.995391
IDPERTET       63.995391
IDPROFCER      63.995391
EDAD           63.525998
cod_inst       59.130769
nom_inst       58.787258
c_ant2         57.889010
c_ant1         37.984595
codpres        37.355181
c_muerte       28.052657
c_dir1         27.719815
area_res       27.559794
asis_med       27.521389
pman_muer      27.521389
seg_social     27.521389
causa_666      27.521389
dtype: float64

In [458]:
missing_count = missing_percent[missing_percent > 0].count()
print("La cantidad de variables con datos faltantes es de: ", missing_count)
without_missing = missing_percent[missing_percent == 0].count()
print("La cantidad de variables sin datos faltantes es de: ", without_missing)
missing_above_70 = missing_percent[missing_percent > 70]
print("La cantidad de variables con más del 70% de datos faltantes es de: ", missing_above_70.count())
missing_below_70 = missing_percent[missing_percent <= 70]
print("La cantidad de variables con menos del 70% de datos faltantes es de: ", missing_below_70.count())


La cantidad de variables con datos faltantes es de:  31
La cantidad de variables sin datos faltantes es de:  22
La cantidad de variables con más del 70% de datos faltantes es de:  13
La cantidad de variables con menos del 70% de datos faltantes es de:  40


In [459]:
color_scale = ["#F8BBD0", "#F48FB1", "#F06292", "#EC407A", "#D81B60"]

fig = px.bar(
    missing_percent, x = missing_percent.index, y = missing_percent.values,
    labels = {'x': 'Variables', 'y': 'Porcentaje de datos Faltantes (%)'},
    title = 'Porcentaje de datos faltantes por variable',
    color = missing_percent.values, color_continuous_scale= color_scale, text = missing_percent.values.round(2))

fig.update_layout(xaxis_title = 'Variables', yaxis_title = 'Porcentaje de datos Faltantes', title_font_size = 20, xaxis_tickangle = -45,
    plot_bgcolor = 'white', paper_bgcolor = 'white',
    font = dict(family = 'Bahnschrift', size = 14, color = 'black'),
    title_font = dict(family = 'Bahnschrift', size = 22, color = 'black'),
    yaxis = dict(range = [0, missing_percent.max() + 10]))

fig.update_traces(textposition ='outside') # Para que el porcentaje se muestre fuera de la barra

fig.show()

COMENTAR

### **Eliminación de variables**


Para optimizar el análisis, eliminamos ciertas variables que no aportan valor significativo. La decisión se tomó con base en los siguientes criterios:  



**1. Variables eliminadas por recomendación de expertos**  
Los profesionales encargados del conjunto de datos recomendaron eliminar estas variables, ya que en este conjunto de datos NO son relevantes para el estudio y están representadas por otras variables:  
(`cau_homol`, `EDAD`, `llave`, `filter_$`, `AÑOL`, `MAMA`)  



**2. Variables eliminadas por más del 75% de datos faltantes**  
Estas variables tienen un porcentaje muy alto de valores nulos, lo que dificulta su uso sin introducir sesgos en el análisis:  
**Observación:** Debido al alto porcentaje de datos faltantes, estas variables no pueden ser imputadas de manera confiable. Además, su eliminación no afecta el análisis, ya que no aportan información relevante o pueden derivarse de otras variables presentes en el conjunto de datos.

| Variable      | Porcentaje de Valores Nulos |
|--------------|----------------------------|
| `codocur`    | 100.00%                     |
| `codmunoc`   | 100.00%                     |
| `c_mcm1`     | 99.85%                      |
| `C_ANT32`    | 99.64%                      |
| `C_DIR12`    | 99.37%                      |
| `C_ANT22`    | 98.98%                      |
| `C_ANT12`    | 98.60%                      |
| `C_PAT2`     | 98.48%                      |
| `SIMUERTEPO` | 98.04%                      |
| `man_muer`   | 93.35%                      |
| `c_ant3`     | 85.70%                      |
| `c_pat1`     | 84.82%                      |
| `OCUPACION`  | 75.80%                      |


**3. Variables que no son relevantes para el estudio**
Estas variables NO serán utilizadas en el enfoque de este estudio y, además, presentan una cantidad significativa de valores nulos:

| Variable     | Motivo de Eliminación                    | Porcentaje de Valores Nulos |
|-------------|-----------------------------------------|----------------------------|
| `a_defun`   | No aporta información relevante        | 0%                         |
| `cod_inst`  | Código de institución poco útil. Alto porcentaje de datos faltantes. Quizás esta información de las instituciones sea antigua debido a las fechas en las que fueron tomadas estas observaciones. | 59.13%                     |
| `nom_inst`  | Ocurre del mismo modo que el código de la institución. No se puede imputar porque no se podría garantizar la institución. | 58.79%                     |
| `MUERTEPORO`| Alto número de faltantes. Adicional a ello, anteriormente la OCUPACIÓN fue eliminada por alto número de faltantes, entonces no tendría sentido analizar la muerte por ocupación sin tener la ocupación. | 63.99%                     |
| `codpres`   | Código no relevante para el análisis   | 37.36%                     |
| `codmunre`  | Código no relevante para el análisis   | 0%                         |
| `area_res`  | Información no necesaria               | 27.56%                     |
| `IDCLASADMI`| No aporta valor significativo y alto porcentaje de datos faltantes | 65.58%                     |
| `cons_exp`  | Código sin valor significativo en el estudio | 0%                         |
| `IDPROFCER` | No aporta valor significativo          | 63.99%                     |
| `quinquenio`| Puede calcularse a partir de la edad. Nos interesa más centrarnos en año y mes  | 0%                         |



In [460]:
data = data.drop(columns=[
    'a_defun','cod_inst', 'nom_inst', 'MUERTEPORO', 'SIMUERTEPO', 'OCUPACION', 
    'codpres','codmunre', 'area_res', 'IDCLASADMI', 'cons_exp', 'man_muer' ,'codocur', 'codmunoc','C_DIR12', 
    'C_ANT12', 'C_ANT22','c_ant3', 'C_ANT32', 'c_pat1', 'C_PAT2',
    'c_mcm1', 'IDPROFCER',  'cau_homol', 'EDAD', 'llave', 'filter_$','AÑOL', 'MAMA', 'quinquenio'
])


In [461]:
data.shape

(46869, 23)

Nótese que la eliminación se realizó de forma exitosa y ahora el conjunto de datos cuenta con solo **23** variables.

### **Renombrado de variables**

In [462]:
data = data.rename(columns={
    'COD_DPTO': 'depto_ocurr',  # Departamento donde ocurrió la defunción (DIVIPOLA)
    'cod_munic': 'munic_ocurr',  # Municipio donde ocurrió la defunción (DIVIPOLA)
    'sit_defun': 'sitio_def',  # Sitio de la defunción
    'ano': 'año_def',  # Año en que ocurrió la defunción
    'mes': 'mes_def',  # Mes en que ocurrió la defunción
    'sexo': 'sexo',  # Sexo del fallecido
    'est_civil': 'estado_civil',  # Estado conyugal del fallecido
    'gru_ed1': 'grupo_edad',  # Agrupación de edades del fallecido
    'nivel_edu': 'nivel_edu',  # Nivel educativo del fallecido
    'IDPERTET': 'grupo_etnico',  # Pertenencia étnica del fallecido
    'codptore': 'rec_etnico' , #De acuerdo con la cultura, pueblo o rasgos físicos, el fallecido
    'CODPTO': 'depto_res',  # Departamento de residencia habitual del fallecido
    'seg_social': 'seg_social',  # Régimen de seguridad social del fallecido
    'IDADMISALU': 'ent_salud_cod',  # Código de la Entidad Administradora en Salud
    'pman_muer': 'prob_muerte',  # Probable manera de muerte
    'c_muerte': 'cert_medica',  # Cómo se determinó la causa de muerte
    'asis_med': 'asistencia_med',  # Recibió asistencia médica antes de fallecer
    'c_dir1': 'causa_directa',  # Causa directa de la defunción
    'c_ant1': 'causa_ant_1',  # Primera causa antecedente
    'c_ant2': 'causa_ant_2',  # Segunda causa antecedente
    'c_bas1': 'causa_basica',  # Código de la causa básica de la defunción
    'causa_666': 'causa_OPS',  # Causa agrupada según la lista OPS 6/67
    'REGION': 'region'  # Región del país donde ocurrió la defunción
})


In [463]:
data.head()

,depto_ocurr,munic_ocurr,sitio_def,año_def,mes_def,sexo,estado_civil,grupo_edad,nivel_edu,grupo_etnico,...,prob_muerte,cert_medica,asistencia_med,causa_directa,causa_ant_1,causa_ant_2,causa_basica,causa_OPS,depto_res,region
0,05,001,1,2014,07,2,1,10,3,6,...,1,2,1,J969,C782,C509,C509,208,05,1
1,05,045,1,1997,08,2,5,10,9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,C509,NaN,05,1
2,05,266,2,1986,04,2,1,10,9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1749,NaN,05,1
3,08,001,1,1989,07,2,2,10,9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1749,NaN,08,2
4,08,001,1,1998,08,2,2,10,3,NaN,...,1,2,1,J969,C780,C509,C509,208,08,2


In [464]:
missing_percent = (data.isnull().sum() / len(data)) * 100
missing_percent = missing_percent.sort_values(ascending=False)
color_scale = ["#F8BBD0", "#F48FB1", "#F06292", "#EC407A", "#D81B60"]

fig = px.bar(
    missing_percent, x = missing_percent.index, y = missing_percent.values,
    labels = {'x': 'Variables', 'y': 'Porcentaje de datos Faltantes (%)'},
    title = 'Porcentaje de datos faltantes por variable',
    color = missing_percent.values, color_continuous_scale= color_scale, text = missing_percent.values.round(2))

fig.update_layout(xaxis_title = 'Variables', yaxis_title = 'Porcentaje de datos Faltantes', title_font_size = 20, xaxis_tickangle = -45,
    plot_bgcolor = 'white', paper_bgcolor = 'white',
    font = dict(family = 'Bahnschrift', size = 14, color = 'black'),
    title_font = dict(family = 'Bahnschrift', size = 22, color = 'black'),
    yaxis = dict(range = [0, missing_percent.max() + 10]))

fig.update_traces(textposition ='outside') # Para que el porcentaje se muestre fuera de la barra

fig.show()

### **Re-estructuración de categorías por variables**

#### **Variable: sexo**

In [465]:
data['sexo'].replace('2', 'Femenino', inplace = True)

#### **Variable: Mes de defunción**

In [466]:
mes_dict_str = {
    "01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril",
    "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto",
    "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre"
}
data['mes_def'] = data['mes_def'].astype(str).str.zfill(2).replace(mes_dict_str)


#### **Variable: Estado civil**

In [467]:
estado_civil_dict = {"1": "Soltero", "2": "Casado", "3": "Viudo", "4": "Unión Libre, Divorciado/Otro", "5": "Sin información"}
data['estado_civil'] = data['estado_civil'].replace(estado_civil_dict)


#### **Variable: Sitio de defunción**

In [468]:
sitio_def_dict = {"1": "Hospital o Clínica", "2": "Casa", "3": "Otro Sitio", "4": "Sin información"}
data['sitio_def'] = data['sitio_def'].astype(str).replace(sitio_def_dict)

#### **Variable: Nivel educativo**

In [469]:
nivel_edu_dict = {"1": "Preescolar", "2": "Primaria", "3": "Secundaria", "4": "Superior", "5": "Ninguno", "9": "Sin información"}
data['nivel_edu'] = data['nivel_edu'].astype(str).replace(nivel_edu_dict)


#### **Variable: Región**

In [470]:
data['region'].value_counts(dropna = False)

region
1    29448
3     8509
2     7620
4      939
5      353
Name: count, dtype: int64

In [471]:
region_dict = {"1": "ANDINA", "2": "ATLANTICA", "3": "PACIFICA", "4": "ORINOQUIA", "5": "PACIFICA"}
data['region'] = data['region'].astype(str).replace(region_dict)


#### **Variable: Causa agrupada con base en la Lista 6/67 de la OP**

In [472]:
data['causa_OPS'].value_counts(dropna = False)

causa_OPS
208    33970
NaN    12899
Name: count, dtype: int64

In [473]:
data['causa_OPS'].fillna('208', inplace = True) 

In [474]:
data['causa_OPS'].replace('208', 'Tumor Maligno', inplace = True)

#### **Variable: Asistencia médica**

In [475]:
data['asistencia_med'].fillna('4', inplace = True)
asistencia_med_dict = {"1": "SI", "2": "NO", "3": "Ignorado", "4": "Sin información"}
data['asistencia_med'] = data['asistencia_med'].astype(str).replace(asistencia_med_dict)


#### **Variable: Probable manera de muerte**

In [476]:
data['prob_muerte'].value_counts(dropna = False)

prob_muerte
1      33965
NaN    12899
3          5
Name: count, dtype: int64

In [477]:
data['prob_muerte'].replace('3', np.nan, inplace=True)

In [478]:
data['prob_muerte'].value_counts(dropna = False)
data['prob_muerte'].fillna('Sin información', inplace = True)

In [479]:
prob_muerte_dict = {"1": "Natural", "2": "Violenta", "3": "En estudio"}
data['prob_muerte'] = data['prob_muerte'].astype(str).replace(prob_muerte_dict)


#### **Variable: Grupo de edad**

In [480]:
data['grupo_edad'].value_counts(dropna = False)


grupo_edad
17    8052
18    5938
19    5260
23    4910
16    4702
20    4486
21    3922
22    3238
15    3200
14    1881
13     930
12     267
11      71
10      12
Name: count, dtype: int64

In [481]:
data['grupo_edad'].fillna('25', inplace = True)

In [482]:
grupo_edad_dict = {
    "10": "15-19 años", "11": "20-24 años", "12": "25-29 años",
    "13": "30-34 años", "14": "35-39 años", "15": "40-44 años",
    "16": "45-49 años", "17": "50-54 años", "18": "55-59 años",
    "19": "60-64 años", "20": "65-69 años", "21": "70-74 años",
    "22": "75-79 años", "23": "80-84 años", "24": "85+ años",
    "25": "Edad desconocida"
}

data['grupo_edad'] = data['grupo_edad'].astype(str).replace(grupo_edad_dict)


In [483]:
data['grupo_edad'].value_counts(dropna = False)

grupo_edad
50-54 años    8052
55-59 años    5938
60-64 años    5260
80-84 años    4910
45-49 años    4702
65-69 años    4486
70-74 años    3922
75-79 años    3238
40-44 años    3200
35-39 años    1881
30-34 años     930
25-29 años     267
20-24 años      71
15-19 años      12
Name: count, dtype: int64

#### **Variable: Seguridad social**

In [484]:
data['seg_social'].value_counts(dropna = False)

seg_social
1      17658
NaN    12899
2      11095
3       2706
5        969
4        854
9        567
6        121
Name: count, dtype: int64

In [485]:
data['seg_social'].fillna('9', inplace = True)

In [486]:
seg_social_dict = {
    "1": "Contributivo", "2": "Subsidiado", "3": "Vinculado",
    "4": "Particular", "5": "Otro", "6": "Ignorado",
    "9": "Sin información"
}

data['seg_social'] = data['seg_social'].astype(str).replace(seg_social_dict)


#### **Variable: cert_medica - ¿Cómo se determino la muerte?**

In [487]:
data['cert_medica'].value_counts(dropna = False)

cert_medica
2      28059
NaN    13148
4       4715
9        758
1        122
3         67
Name: count, dtype: int64

In [488]:
data['cert_medica'].fillna('9', inplace = True)

In [489]:
cert_medica_dict = {
    "1": "Necropsia", "2": "Historia clínica", "3": "Pruebas de laboratorio",
    "4": "Interrogatorio a familiares o testigos", "9": "Sin información"
}
data['cert_medica'] = data['cert_medica'].astype(str).replace(cert_medica_dict)

#### **Variable: Código de la entidad administradora en salud a la que pertenecía el fallecido**

In [490]:
data['ent_salud_cod'].value_counts(dropna = False)


ent_salud_cod
NaN    30677
1       8497
2       6375
5        659
9        507
4        110
3         44
Name: count, dtype: int64

In [491]:
data['ent_salud_cod'].fillna('9', inplace = True)

In [492]:
ent_salud_dict = {
    "1": "EPS", "2": "EPS - Subsidiado", "3": "EAPB",
    "4": "ESE", "5": "EESS", "9": "Sin información"}
data['ent_salud_cod'] = data['ent_salud_cod'].astype(str).replace(ent_salud_dict)

#### **Variable: Grupo étnico**

In [493]:
data['grupo_etnico'].value_counts(dropna = False)

grupo_etnico
NaN    29994
6      14549
9       1285
5        894
1         87
2         33
3         21
4          6
Name: count, dtype: int64

In [494]:
data['grupo_etnico'].fillna('9', inplace = True)

In [495]:
grupo_etnico_dict = {
    "1": "Indígena", "2": "Rom (Gitano)", 
    "3": "Raizal del archipiélago de San Andrés y Providencia",
    "4": "Palenquero de San Basilio", 
    "5": "Negro(a), mulato(a), afrocolombiano(a) o afrodescendiente",
    "6": "Ninguno de los anteriores", "9": "Sin información"
}

data['grupo_etnico'] = data['grupo_etnico'].astype(str).replace(grupo_etnico_dict)


#### **Variable: municipio de defunción**

In [496]:
data["municipio"] = data["depto_ocurr"].astype(str) + data["munic_ocurr"].astype(str).str.zfill(3)

## **Conjunto de datos 2: DIVIPOLA**


Este conjunto de datos proporciona los códigos oficiales utilizados para identificar departamentos, municipios y otras divisiones territoriales. Estos códigos sirven como referencia en el Conjunto de Datos 1 para estandarizar la información geográfica y facilitar su análisis.

In [497]:
column_types = {
    "Código Departamento": "object",
    "Código Municipio": "object",
    "Código Centro Poblado": "object",
    "Nombre Departamento": "object",
    "Nombre Municipio": "object",
    "Nombre Centro Poblado	": "object",
    "Tipo": "object"
}
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/Divipola.csv"
data2 = pd.read_csv(url, delimiter = ",", na_values = [" "], dtype = column_types)
data2.head(15)

,Código Departamento,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo
0,05,05001,05001000,ANTIOQUIA,MEDELLÍN,MEDELLÍN,CM
1,05,05001,05001001,ANTIOQUIA,MEDELLÍN,PALMITAS,C
2,05,05001,05001004,ANTIOQUIA,MEDELLÍN,SANTA ELENA,C
3,05,05001,05001005,ANTIOQUIA,MEDELLÍN,PEDREGAL ALTO,IPM
4,05,05001,05001009,ANTIOQUIA,MEDELLÍN,ALTAVISTA,C
5,05,05001,05001010,ANTIOQUIA,MEDELLÍN,AGUAS FRÍAS,CP
6,05,05001,05001012,ANTIOQUIA,MEDELLÍN,LA LOMA,CP
7,05,05001,05001013,ANTIOQUIA,MEDELLÍN,SAN JOSÉ DEL MANZANILLO,CP
8,05,05001,05001014,ANTIOQUIA,MEDELLÍN,BARRO BLANCO,CP
9,05,05001,05001015,ANTIOQUIA,MEDELLÍN,EL CERRO,CP


In [498]:
data2.shape

(9205, 7)

In [499]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Código Departamento    9205 non-null   object
 1   Código Municipio       9205 non-null   object
 2   Código Centro Poblado  9205 non-null   object
 3   Nombre Departamento    9205 non-null   object
 4   Nombre Municipio       9205 non-null   object
 5   Nombre Centro Poblado  9205 non-null   object
 6   Tipo                   9205 non-null   object
dtypes: object(7)
memory usage: 503.5+ KB


Nótese que se cuenta con un conjunto de datos bastante completo. 

## **Conjunto de datos final**

In [ ]:
departamento_dict = dict(zip(data2['Código Departamento'], data2['Nombre Departamento']))
municipio_dict = dict(zip(data2['Código Municipio'], data2['Nombre Municipio']))
data['Nombre_Departamento'] = data['depto_ocurr'].map(departamento_dict)
data['Nombre_Municipio'] = data['municipio'].map(municipio_dict)
data['depto_res'] = data['depto_res'].map(departamento_dict)
data['rec_etnico'] = data['rec_etnico'].map(departamento_dict)
data = data.drop(columns = ['munic_ocurr', 'municipio', 'depto_ocurr']) 
data.head()


In [502]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46869 entries, 0 to 46868
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   depto_ocurr          46869 non-null  object
 1   sitio_def            46869 non-null  object
 2   año_def              46869 non-null  Int64 
 3   mes_def              46869 non-null  object
 4   sexo                 46869 non-null  object
 5   estado_civil         46869 non-null  object
 6   grupo_edad           46869 non-null  object
 7   nivel_edu            46869 non-null  object
 8   grupo_etnico         46869 non-null  object
 9   rec_etnico           17066 non-null  object
 10  seg_social           46869 non-null  object
 11  ent_salud_cod        46869 non-null  object
 12  prob_muerte          46869 non-null  object
 13  cert_medica          46869 non-null  object
 14  asistencia_med       46869 non-null  object
 15  causa_directa        33877 non-null  object
 16  caus

In [503]:
columnas_ordenadas = [
    'Nombre_Departamento', 'Nombre_Municipio', 'sitio_def', 'año_def', 
    'mes_def', 'sexo', 'estado_civil', 'grupo_edad', 'nivel_edu', 'grupo_etnico','rec_etnico',
    'depto_res', 'seg_social', 'ent_salud_cod', 'prob_muerte', 'cert_medica', 'asistencia_med',
    'causa_directa', 'causa_ant_1', 'causa_ant_2', 'causa_basica', 'causa_OPS', 'region', 'depto_ocurr', 'municipio'
]
data_analisis = data[columnas_ordenadas]

In [504]:
data_analisis.head()

,Nombre_Departamento,Nombre_Municipio,sitio_def,año_def,mes_def,sexo,estado_civil,grupo_edad,nivel_edu,grupo_etnico,...,cert_medica,asistencia_med,causa_directa,causa_ant_1,causa_ant_2,causa_basica,causa_OPS,region,depto_ocurr,municipio
0,ANTIOQUIA,MEDELLÍN,Hospital o Clínica,2014,Julio,Femenino,Soltero,15-19 años,Secundaria,Ninguno de los anteriores,...,Historia clínica,SI,J969,C782,C509,C509,Tumor Maligno,ANDINA,05,05001
1,ANTIOQUIA,APARTADÓ,Hospital o Clínica,1997,Agosto,Femenino,Sin información,15-19 años,Sin información,Sin información,...,Sin información,Sin información,NaN,NaN,NaN,C509,Tumor Maligno,ANDINA,05,05045
2,ANTIOQUIA,ENVIGADO,Casa,1986,Abril,Femenino,Soltero,15-19 años,Sin información,Sin información,...,Sin información,Sin información,NaN,NaN,NaN,1749,Tumor Maligno,ANDINA,05,05266
3,ATLÁNTICO,BARRANQUILLA,Hospital o Clínica,1989,Julio,Femenino,Casado,15-19 años,Sin información,Sin información,...,Sin información,Sin información,NaN,NaN,NaN,1749,Tumor Maligno,ATLANTICA,08,08001
4,ATLÁNTICO,BARRANQUILLA,Hospital o Clínica,1998,Agosto,Femenino,Casado,15-19 años,Secundaria,Sin información,...,Historia clínica,SI,J969,C780,C509,C509,Tumor Maligno,ATLANTICA,08,08001


In [507]:
data_analisis.to_excel('C:\\Users\\kamac\\OneDrive\\Desktop\\SeminarioInvestigativoUN\\docs\\data_analisis.xlsx', index=False, engine='openpyxl')

```{note}

En el siguiente enlace se encuentra el diccionario de variables utilizado en este proyecto:

[Diccionario de Variables](https://kmarcela11.github.io/ProyectoFinal_SeminarioInvestigativo/variables.html)

Este diccionario incluye una descripción detallada de cada una de las variables presentes en el conjunto de datos, facilitando su comprensión y el análisis del proyecto.

```